In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from munkres import Munkres, print_matrix
from scipy.spatial import distance

In [ ]:
folder = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1062_play_1'
poses = sorted_alpha(folder)

In [ ]:
frame_no = [int((x.split("."))[0].split("_")[1]) for x in poses]

In [ ]:
def get_skeletons(frame_no, detection):
    """function that reads a detections frame, checks if necks are present, and returns the skeletons"""
    data = pd.read_csv(detection, index_col=0)
   # if len(data.columns)>9: # drop the fourth skeleton if detected because there can't be 4 people
    #    data.drop(columns=['x_4', 'y_4', 'prob_4'], inplace=True)
    ## first check if it contains the neck 
    temp_data = data
    for i in range(0, len(data.columns),3): # drop skeletons without neck
        if np.isnan(data[data.columns[i]].loc[1]) == True:
            temp_data = temp_data.drop([data.columns[i],data.columns[i+1], data.columns[i+2]], axis=1) ## drop skeletons with missing neck
    data=temp_data
    if len(data.columns) < 3: ## if not enough skeletons with neck present return 0 
        return(None)
    else:
        skeletons=[]
        ## remove probs columns
        for col in data.columns:
            if data[col].name[0:3] == 'pro':
                data=data.drop([col], axis=1)
     # return every skeleton now  
        for x,y, in zip(data.columns[::2], data.columns[1::2]):
            skeletons.append(data[[x,y]].to_numpy())
        return(skeletons) ## this is a list of skeletons(stacked)
        
    

In [ ]:
def dist_matrix(prev_detections, new_detect):
    """calculates distances between each new detection and the previous detections from each track"""
    """returns a matrix of shape n x m where n is len(prev_detections) and m is len(new_detect)"""
    tracklets = [x['skeleton'][-1][1] for x in prev_detections if x['skeleton']!=[]]
    new_detect = [x[1] for x in new_detect]
    return(distance.cdist(tracklets, new_detect, 'euclidean'))

In [ ]:
def assignment(distance_matrix):
    """does optimal assignment of detections to tracks using hungarian algorithm"""
    m = Munkres()
    if distance_matrix.shape[0]>distance_matrix.shape[1]:
        matrix = np.transpose(distance_matrix)
        indexes = m.compute(matrix)
        indexes = [x[::-1] for x in indexes]
    else:
        matrix = m.pad_matrix(distance_matrix) # pads the matrix for when there are more detections than initialized tracks
        indexes = m.compute(matrix)
    return(indexes)


In [ ]:
def missing_elements(L):
    start, end = L[0], L[-1]
    return sorted(set(range(start, end + 1)).difference(L))

In [ ]:
def insert_position(position, list1, list2): ## position is the upper bound
    return list1[:position] + list2 + list1[position:]

In [ ]:
def save_tracks(path, folder, tracks):
    for track,k in zip(tracks, range(len(tracks))):
        if len(track['frame'])<11:
            continue
        track['skeleton'] = savgol_filter(track['skeleton'], 11, 2, axis=0)
        frames = track['frame']
        joints = [x for x in range(0,18)]
        frames_index = [x for x in  frames for joint in joints]
        joints_index = joints*len(track['skeleton'])
        track_df = pd.DataFrame((np.vstack(track['skeleton'])), index = [frames_index, joints_index],columns=['x', 'y'])
        track_df = track_df.rename_axis(['frame','joint'])
        track_df.to_csv(os.path.join(path, folder,'track_'+str(k+1)+'.csv'), na_rep=np.nan)
    return('success')

In [ ]:
def find_min_round(coord):
    coord = np.array(coord)
    x = round(np.min(coord[np.isfinite(coord)]))
    return(x)

def find_max_round(coord):
    coord = np.array(coord)
    x = round(np.max(coord[np.isfinite(coord)]))
    return(x)

In [ ]:
a = [2,3]
a[:5]

In [ ]:
track_1, track_2= {'frame':[], 'skeleton':[]}, {'frame':[], 'skeleton':[]}
tracks = [track_1, track_2]
frames = []
for pose, no in zip(poses, frame_no):
    print(no)
    frames.append(frame_no)
    detect = get_skeletons(no, os.path.join(folder, pose))
    if detect == None: continue
   # if track_1['frame'] == []: # initializing the first track 
    while track_1['frame'] == [] and track_2['frame'] ==[]:
        for detection in range(len(detect[:2])): ## restricting to only 2 detections
            tracks[detection]['skeleton'].append(detect[detection]) ## in the first frame initalize the tracks
            tracks[detection]['frame'].append(no)
       # continue
    matrix = dist_matrix(tracks, detect) #creates a matrix of distances between previous detections and new detections
    indexes = assignment(matrix) # get the indices of which detections belongs to which track
    for row, column in indexes[:len(tracks)]: ## the indexes could be padded so might have 3 rows but only 2 tracks 
        tracks[row]['skeleton'].append(detect[column])
        tracks[row]['frame'].append(no)
    ## interpolation
    for track in tracks:
        if track['frame']==[] or len(track['frame'])<2:continue

        gaps = missing_elements([track['frame'][-2], track['frame'][-1]]) ## the gaps are the missing frame numbers not indexes
        if len(gaps) > 0 and len(gaps) <50:
            f = interp1d(track['frame'], track['skeleton'], axis=0)
            to_insert = f(gaps).tolist()
            track['skeleton'] = insert_position(track['frame'].index(gaps[-1]+1), 
                                                track['skeleton'], to_insert)
            track['frame'] = insert_position(track['frame'].index(gaps[-1]+1), track['frame'], gaps)
            
            

In [ ]:
track_2

## for all 

In [ ]:
path = '/Users/andrei-macpro/Documents/Data/pose/play_openpose'
folders = sorted_alpha(path)

In [ ]:
for folder in tqdm(folders[11:]):
    new_path = '/Users/andrei-macpro/Documents/Data/tracking/play_tracking_2'
    os.mkdir(os.path.join(new_path, folder))
    os.chdir(os.path.join(path,folder))
    detections = sorted_alpha(os.path.join(path,folder))
    frame_no = [int((x.split("."))[0].split("_")[1]) for x in detections]
    track_1, track_2 = {'frame':[], 'skeleton':[]}, {'frame':[], 'skeleton':[]}
    tracks = [track_1, track_2]
    frames = []
    for pose, no in zip(detections, frame_no):
        detect = get_skeletons(no, os.path.join(path,folder, pose))
        if detect == None: continue
        while track_1['frame'] == [] and track_2['frame'] ==[]: # initializing the first 2 tracks
            for detection in range(len(detect[:2])): ## if there are more than 2 tracks can change max detection number
                tracks[detection]['skeleton'].append(detect[detection]) ## in the first frame initalize the tracks
                tracks[detection]['frame'].append(no)
        matrix = dist_matrix(tracks, detect) #creates a matrix of distances between previous detections and new detections
        indexes = assignment(matrix) # get the indices of which detections belongs to which track
    
        for row, column in indexes[:len(tracks)]:
            tracks[row]['skeleton'].append(detect[column])
            tracks[row]['frame'].append(no)
        ## interpolation
        for track in tracks:
            if track['frame']==[] or len(track['frame'])<2:continue
    
            gaps = missing_elements([track['frame'][-2], track['frame'][-1]]) ## the gaps are the missing frame numbers not indexes
            if len(gaps) > 0 and len(gaps) <50:
                f = interp1d(track['frame'], track['skeleton'], axis=0)
                to_insert = f(gaps).tolist()
                track['skeleton'] = insert_position(track['frame'].index(gaps[-1]+1), 
                                                    track['skeleton'], to_insert)
                track['frame'] = insert_position(track['frame'].index(gaps[-1]+1), track['frame'], gaps)
    save_tracks(new_path,folder,tracks)
print('done')

In [ ]:
folders[11:]

In [ ]:
frames_index = [x for x in  frames for joint in joints]
joints_index = joints*len(track_1)

In [ ]:
track_1_df = pd.DataFrame((np.vstack(track_1)), index = [frames_index, joints_index],columns=['x', 'y'])

In [ ]:
track_1_df = track_1_df.rename_axis(['frame','joint'])

In [ ]:
track_index = track_1_df.index.get_level_values('frame')
for x in track_index:
    print(track_1_df.loc[x])
    break

In [ ]:
path_video = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1043_play_1.mp4'
video_length = frame_count(path_video, manual=True)

In [ ]:
cap = cv2.VideoCapture(path_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')# note the lower case
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
out = cv2.VideoWriter('1043_play_1_track.mp4', fourcc, 25.0, (int(width),int(height)))

In [ ]:
color = (100, 149, 237)
thickness = 2
for frame in np.unique(track_1_df.index.get_level_values('frame')):
    cap.set(1,frame)
    ret, image = cap.read()
    human = cv2.rectangle(image, (find_min_round(track_1_df.loc[frame]['x']), find_min_round(track_1_df.loc[frame]['y'])),  
                            (find_max_round(track_1_df.loc[frame]['x']), find_max_round(track_1_df.loc[frame]['y'])), color, thickness)
  #  cv2.putText(human, f"frame: {frame}" , (10, 15),
   #         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    out.write(human)

cap.release()
out.release()
cv2.destroyAllWindows()     

In [ ]:
np.unique(track_1_df.index.get_level_values('frame'))


In [ ]:
array[np.isfinite(array)]

In [ ]:
np.max(array)

In [ ]:
### path = '/Users/andrei-macpro/Documents/Data/tracking/play_tracking_2/1043_play/track_1.csv'
data = pd.read_csv(path, index_col=[0,1])
data

In [ ]:
np.unique(data.index.get_level_values('frame'))

In [ ]:
path = '/Users/andrei-macpro/Documents/Data/tracking/play_tracking_2'
folders = sorted_alpha(path)
path_videos ='/Users/andrei-macpro/Documents/Data/videos/play_videos'
np.random.seed(42) 
new_path = '/Users/andrei-macpro/Documents/Data/tracking/eval_play_tracking_2'
for folder in tqdm(folders): 
    cap = cv2.VideoCapture(os.path.join(path_videos, folder+'.mp4' ))
    new_p = os.path.join(new_path, folder)
    os.makedirs(new_p)
    for track in sorted_alpha(os.path.join(path,folder)):
        new_dir = os.path.join(new_path, folder,track)
        os.makedirs(new_dir)
        data = pd.read_csv(os.path.join(path,folder,track), index_col=[0,1])
        for frame in np.random.choice(np.unique(data.index.get_level_values('frame')), size=1000, replace=False):

            df = data.loc[frame].round(0)
            cap.set(1, frame)
            ret,image = cap.read()
            if ret==False:
                continue
            img = image[find_min_round(df['y']):find_max_round(df['y']),find_min_round(df['x']):find_max_round(df['x'])]
            if img.shape[0]>0 and img.shape[1]>0:
                cv2.imwrite(os.path.join(new_dir,str(frame)+'.png'), img)

    

In [ ]:
cap = cv2.VideoCapture(os.path.join(path_videos, folders[1], '.mp4'))

cap.set(1, 100)
ret,image = cap.read()
print(image)

In [ ]:
folders

In [ ]:
# delete the tracks that are very small 
path = '/Users/andrei-macpro/Documents/Data/tracking/play_tracking_2'
folders = sorted_alpha(path)


In [ ]:
for folder in folders:
    print(folder)
    l = []
    files = sorted_alpha(os.path.join(path,folder))

    for file in files:
        df = pd.read_csv(os.path.join(path,folder,file),index_col=[0,1])
        l.append(len(np.unique(df.index.get_level_values('frame'))))

    if min(l) < max(l)*20/100:

        os.remove(os.path.join(path,folder,files[l.index(min(l))]))
        
        
        

In [ ]:
a = [1,2,3]

In [ ]:
for x in a: 
    l = []
    l.append(1)
    print(l)